# 05 - Conversor e Filtro para Submissão Final

**🎯 Objetivo deste Notebook:**

Este notebook serve como um passo final e documentado para formatar o arquivo de previsão gerado pelo `04-Final-Pipeline.ipynb`. Sua principal função é ler o arquivo Parquet completo, que contém previsões para todas as combinações de PDV/SKU (incluindo previsões de venda zero), e convertê-lo para o formato final.

**✅ Processo de Conversão:**

1.  **Carregar Previsões:** Lê o arquivo `submission.parquet` da pasta `/submissions`, que contém o output completo do modelo.
2.  **Filtrar Vendas Zero:** Remove todas as linhas onde a coluna `quantidade` é igual a 0. Esta é uma exigência da plataforma, que limita o número de linhas do arquivo de submissão.
3.  **Renomear Colunas (Validação):** Garante que os nomes das colunas (`semana`, `pdv`, `produto`, `quantidade`) estão exatamente como o especificado no regulamento.
4.  **Salvar em CSV:** Exporta o `DataFrame` filtrado para um novo arquivo CSV (`submission_final.csv`) usando `;` como separador e encoding `UTF-8`, pronto para o upload.

In [ ]:
import pandas as pd
import os

print('📚 Bibliotecas carregadas com sucesso!')

## 1. Definir Caminhos dos Arquivos

In [ ]:
# Caminho para o arquivo de entrada gerado pelo pipeline 04
input_parquet_path = '../submissions/submission.parquet'

# Caminho para o arquivo de saída final que será submetido
output_csv_path = '../submissions/submission_final.csv'

print(f"Arquivo de entrada: {input_parquet_path}")
print(f"Arquivo de saída: {output_csv_path}")

## 2. Carregar e Processar o Arquivo de Previsão

In [ ]:
print(f"🔄 Carregando o arquivo Parquet de previsões completas...")
if not os.path.exists(input_parquet_path):
    print(f"❌ Erro: Arquivo '{input_parquet_path}' não encontrado.")
    print("Certifique-se de executar o notebook '04-Final-Pipeline.ipynb' primeiro.")
else:
    # Carregar o DataFrame
    full_predictions_df = pd.read_parquet(input_parquet_path)
    print(f"✅ Arquivo carregado com sucesso. Shape original: {full_predictions_df.shape}")
    
    # --- PASSO 1: Garantir que os nomes das colunas estão corretos ---
    # (Esta é uma validação extra para garantir a compatibilidade)
    rename_map = {
        'pdv_id': 'pdv',
        'produto_id': 'produto'
    }
    columns_to_rename = [col for col in rename_map if col in full_predictions_df.columns]
    if columns_to_rename:
        print(f"Renomeando colunas: {columns_to_rename}")
        full_predictions_df.rename(columns=rename_map, inplace=True)
    
    # --- PASSO 2: Filtrar as linhas com previsão de quantidade zero ---
    print(f"\n🔧 Filtrando para manter apenas previsões com 'quantidade' > 0...")
    filtered_df = full_predictions_df[full_predictions_df['quantidade'] > 0].copy()
    
    linhas_removidas = len(full_predictions_df) - len(filtered_df)
    print(f"   • Linhas removidas: {linhas_removidas:,}")
    print(f"   • Shape final: {filtered_df.shape}")
    
    # --- PASSO 3: Validar os tipos de dados ---
    print("\n✔️ Validando e ajustando os tipos de dados para inteiros...")
    filtered_df['semana'] = filtered_df['semana'].astype(int)
    filtered_df['pdv'] = filtered_df['pdv'].astype(int)
    filtered_df['produto'] = filtered_df['produto'].astype(int)
    filtered_df['quantidade'] = filtered_df['quantidade'].astype(int)

    # --- PASSO 4: Salvar o arquivo CSV final ---
    print(f"\n💾 Salvando arquivo CSV final em '{output_csv_path}'...")
    filtered_df.to_csv(
        output_csv_path,
        sep=';',           # Usando ponto e vírgula como separador
        index=False,       # Não salvar o índice do DataFrame
        encoding='utf-8'   # Garantir o encoding correto
    )
    
    print("\n🎉 Processo concluído! O arquivo 'submission_final.csv' está pronto para ser enviado.")
    
    # --- PASSO 5: Exibir um preview do arquivo final ---
    print("\n👀 Preview do DataFrame final:")
    display(filtered_df.head(10))